** This notebook assumes ... **

1) Basic familiarity with gwsurrogate as covered in basics.ipynb 

2) You've downloaded a numerical relativity surrogate. To see all available surrogates, check the surrogate repository with the command "gws.surrogate_repo.list()"

In [ ]:
### setup paths used throughout this notebook ###
import sys
path_to_gws = '/home/scott/Repos/GitRepos/Codes/gwsurrogate/'
sys.path.append(path_to_gws)

%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
import gwsurrogate as gws
import gwsurrogate.gwtools as gwtools

In [ ]:
### load the surrogate data ###
path_to_surrogate = path_to_gws+'surrogate_downloads/SpEC_q1_10_NoSpin_nu5thDegPoly_exclude_2_0.h5'
spec = gws.EvaluateSurrogate(path_to_surrogate, ell_m=[(2,2), (3,3)]) # load two modes only

# Lesson 1: Simple evaluations

In [ ]:
# Evaluate and plot the 2,2 mode. 
# By default, the modes are evaluated on the sphere, and negative modes are generated from 
# known relationships. So we need to set both options to false to get only the (2,2) mode.
modes, times, hp, hc = spec(q=1.7, ell=[2], m=[2], mode_sum=False, fake_neg_modes=False)
print 'You have evaluated the (%i,%i) mode'%(modes[0][0],modes[0][1])

gwtools.plot_pretty(times, [hp, hc],fignum=1)
plt.plot(times,gwtools.amp(hp+1j*hc),'r')
plt.title('The (%i,%i) mode'%(modes[0][0],modes[0][1]))
plt.xlabel('t/M ')
plt.show()

In [ ]:
# generating both the (3,3) and (3,-3) modes is easy!
modes, times, hp, hc = spec(q=1.7, ell=[3], m=[3], mode_sum=False)
print "Evaluated modes =", modes

gwtools.plot_pretty(times, [hp[:,0], hc[:,0]],fignum=2)
plt.plot(times,gwtools.amp(hp[:,0]+1j*hc[:,0]),'r')
plt.xlabel('t/M ')
plt.title('The (%i,%i) mode'%(modes[0][0],modes[0][1]))

gwtools.plot_pretty(times, [hp[:,1], hc[:,1]],fignum=3)
plt.plot(times,gwtools.amp(hp[:,1]+1j*hc[:,1]),'r')
plt.title('The (%i,%i) mode'%(modes[1][0],modes[1][1]))
plt.xlabel('t/M ')
plt.show()

In [ ]:
# Trying to evaluate a mode which doesn't exist throws a warning
spec(q=1.7, ell=[3], m=[2], mode_sum=False, fake_neg_modes=False)

# Lesson 2: Physical waveforms

In [ ]:
# load all the modes except the 2,0 mode, which is excluded by default (See http://arxiv.org/abs/1502.07758 for why)
spec = gws.EvaluateSurrogate(path_to_surrogate)

In [ ]:
# Evaluate the (2,2) mode for physical input values
M     = 115.0 # units of solar masses 
q     = 1.0
theta = np.pi/3.0
phi   = np.pi/3.0
dist  = 1.0 # units of megaparsecs
fmin  = 10.0 # units of hz

# The surrogate evaluation is NOT long enough to achieve a starting frequency of 10hz
modes,times,hp,hc = spec(M=M,q=q,dist=dist,theta=theta,phi=phi,ell=[2],m=[2],mode_sum=False,fake_neg_modes=False,f_low=fmin)

In [ ]:
# We'll be safer with 15 hz 
fmin = 15
modes,times,hp,hc = spec(M=M,q=q,dist=dist,theta=theta,phi=phi,ell=[2],m=[2],mode_sum=False,fake_neg_modes=False,f_low=fmin)

gwtools.plot_pretty(times, [hp, hc])
plt.plot(times,gwtools.amp(hp+1j*hc),'r')
plt.xlabel('t (seconds)')
plt.title('The (%i,%i) mode'%(modes[0][0],modes[0][1]))
plt.show()

In [ ]:
# Lets evaluate the (2,2) and (2,-2) at phi = theta = pi/3 on the sphere
times,hp,hc = spec(M=M,q=q,dist=dist,theta=theta,phi=phi,ell=[2],m=[2],f_low=fmin)

gwtools.plot_pretty(times, [hp, hc])
plt.plot(times,gwtools.amp(hp+1j*hc),'r')
plt.xlabel('t (seconds)')
plt.show()